#### ESSES DADOS SERVIRÃO COMO PONTO DE PARTIDA PARA A OTIMIZAÇÃO DO PORTFÓLIO. DURANTE O PROCESSO DE OTIMIZAÇÃO, ESSAS ALOCAÇÕES PODEM SER AJUSTADAS PARA BUSCAR UMA COMBINAÇÃO QUE MAXIMIZE O RETORNO GLOBAL, LEVANDO EM CONSIDERAÇÃO AS RESTRIÇÕES E OBJETIVOS DEFINIDOS.

| Ativo           | Tipo de Ativo | Retorno Anual (%) | Volatilidade (%) | Alocação Atual (%) |
| --------------- | ------------- | ----------------- | ---------------- | ------------------ |
| Ações Flamengo  | Ações         | 12                | 20               | 30                 |
| Títulos Gercom  | Títulos       | 6                 | 8                | 40                 |
| Fundo Mútuo CNPQ | Fundo Mútuo   | 8                 | 15               | 30                 |

AÇÕES DO FLAMENGO:

- Tipo de Ativo: Ações 
- Retorno Anual: 12% 
- Volatilidade: 20% 
- Alocação Atual: 30% 


TÍTULOS GERCOM:

* Tipo de Ativo: Títulos
* Retorno Anual: 6%
* Volatilidade: 8%
* Alocação Atual: 40%

FUNDO MÚTUO CNPQ:

* Tipo de Ativo: Fundo Mútuo
* Retorno Anual: 8%
* Volatilidade: 15%
* Alocação Atual: 30%

##### VAMOS MODELAR O PROBLEMA DE OTIMIZAÇÃO PORTFÓLIO. PARA ISSO, UTILIZAREMOS A TEORIA CLÁSSICA DE OTIMIZAÇÃO DE PORTFÓLIO, QUE BUSCA MAXIMIZAR O RETORNO ESPERADO SUJEITO A RESTRIÇÕES DE RISCO
VÁRIAVEIS DE DECISÃO
- $x_1$: A alocação de ativos em Ações Flamengo
- $x_2$: A alocação de ativos em Títulos Gercom
- $x_3$: A alocação de ativos no Fundo Mútuo CNPQ
##### OBJETIVO: 
Maximizar $ \sum _{i=1}^3 retorno_1 \cdot x_i$  
#### Restrições:
1. A soma das alocações deve ser igual a 1:
- $\sum _{i=1}^3x_1=\:1$
2. Restrições de alocação entre 0% e 100% para cada ativo:
- $ 0\le \:x_1\le \:1,\:para\:i\:=\:1,2,3 $
3. Restrições de volatilidade:
- $\sum _{i=1}^3Volatilidade_i\:\cdot x_1\:\le \:Limite\:de\:volatilidade$
##### Parâmetros:
- $Retorno_i$: Retorno anual do ativo  $_i $
- $Volatilidade_i$: Volatilidade anual do ativo   $_i$
- Limite de Volatilidade: Limite aceitável de volatilidade no portfólio

##### Este é um modelo simplificado, e em cenários reais, outros fatores como custos de transação, limites mínimos e máximos de alocação, e restrições específicas do investidor podem ser adicionados ao modelo. Agora, você pode utilizar métodos de otimização clássica, como a Programação Linear, para resolver esse problema e encontrar as alocações ótimas que maximizam o retorno sujeito às restrições dadas.

In [1]:
from scipy.optimize import minimize

# Dados do portfólio
retorno = [0.12, 0.06, 0.08]  # Retorno anual dos ativos
volatilidade = [0.20, 0.08, 0.15]  # Volatilidade anual dos ativos

# Restrições
limite_volatilidade = 0.18  # Limite de volatilidade desejado

# Função objetivo a ser maximizada
def funcao_objetivo(alocacoes):
    return -sum(retorno * alocacoes)

# Restrições
restricoes = (
    {"type": "eq", "fun": lambda alocacoes: sum(alocacoes) - 1},  # Soma das alocações igual a 1
    {"type": "ineq", "fun": lambda alocacoes: limite_volatilidade - sum(volatilidade * alocacoes)}  # Restrição de volatilidade
)

# Limites das alocações (entre 0% e 100% para cada ativo)
limites_alocacao = [(0, 1) for _ in retorno]

# Chute inicial
chute_inicial = [1 / len(retorno) for _ in retorno]

# Otimização
resultado = minimize(funcao_objetivo, chute_inicial, method="SLSQP", bounds=limites_alocacao, constraints=restricoes)

# Exibir resultados
alocacoes_otimizadas = resultado.x
retorno_otimizado = -resultado.fun

print("Alocações otimizadas:", alocacoes_otimizadas)
print("Retorno máximo esperado:", retorno_otimizado)


Alocações otimizadas: [0.83333333 0.16666667 0.        ]
Retorno máximo esperado: 0.10999999979872059


##### A função objetivo QUBO será então formulada como:
 - minimize $\left( \sum_{i=1}^3 \sum_{j=1}^3 \text{Retorno}_1 \cdot x_i \cdot x_j \right)$
##### sujeita às restrições:
- $\sum_{i=1}^3 x_i = 1$
- $\sum_{i=1}^3 \text{Volatilidade}_i \cdot x_i \leq \text{Limite de Volatilidade}$



In [15]:
from dimod import BinaryQuadraticModel
from neal import SimulatedAnnealingSampler

# Dados do portfólio
retorno = [0.12, 0.06, 0.08]  # Retorno anual dos ativos
volatilidade = [0.20, 0.08, 0.15]  # Volatilidade anual dos ativos

# Restrições
limite_volatilidade = 0.18  # Limite de volatilidade desejado

# Função objetivo a ser maximizada
def funcao_objetivo(alocacoes):
    return -sum(retorno * alocacoes)

# Restrições
restricoes = (
    {"type": "eq", "fun": lambda alocacoes: sum(alocacoes) - 1},  # Soma das alocações igual a 1
    {"type": "ineq", "fun": lambda alocacoes: limite_volatilidade - sum(volatilidade * alocacoes)}  # Restrição de volatilidade
)

# Limites das alocações (entre 0% e 100% para cada ativo)
limites_alocacao = [(0, 1) for _ in retorno]

# Chute inicial
chute_inicial = [1 / len(retorno) for _ in retorno]

# Otimização
bqm = BinaryQuadraticModel('BINARY')
for i in range(len(retorno)):
    bqm.add_variable(i, -retorno[i])  # Maximizar retorno
    for j in range(i+1, len(retorno)):
        bqm.add_interaction(i, j, 2*limite_volatilidade*volatilidade[i]*volatilidade[j])  # Restrição de volatilidade

sampler = SimulatedAnnealingSampler()
amostras = sampler.sample(bqm)

# Exibir resultados
alocacoes_otimizadas = amostras.first.sample
retorno_otimizado = -amostras.first.energy

print("Alocações otimizadas:", alocacoes_otimizadas)
print("Retorno máximo esperado:", retorno_otimizado)


Alocações otimizadas: {0: 1, 1: 1, 2: 1}
Retorno máximo esperado: 0.23912000000000003


In [17]:
from scipy.optimize import minimize

# Dados do portfólio
retorno = [0.12, 0.06, 0.08]  # Retorno anual dos ativos
volatilidade = [0.20, 0.08, 0.15]  # Volatilidade anual dos ativos

# Restrições
limite_volatilidade = 0.18  # Limite de volatilidade desejado

# Função objetivo a ser maximizada
def funcao_objetivo(alocacoes):
    return -sum(retorno * alocacoes)

# Restrições
restricoes = (
    {"type": "eq", "fun": lambda alocacoes: sum(alocacoes) - 1},  # Soma das alocações igual a 1
    {"type": "ineq", "fun": lambda alocacoes: limite_volatilidade - sum(volatilidade * alocacoes)}  # Restrição de volatilidade
)

# Limites das alocações (entre 0% e 100% para cada ativo)
limites_alocacao = [(0, 1) for _ in retorno]

# Chute inicial
chute_inicial = [1 / len(retorno) for _ in retorno]

# Otimização
resultado = minimize(funcao_objetivo, chute_inicial, method="SLSQP", bounds=limites_alocacao, constraints=restricoes)

# Exibir resultados
alocacoes_otimizadas = resultado.x
retorno_otimizado = -resultado.fun

print("Alocações otimizadas:", alocacoes_otimizadas)
print("Retorno máximo esperado:", retorno_otimizado)


Alocações otimizadas: [0.83333333 0.16666667 0.        ]
Retorno máximo esperado: 0.10999999979872059


In [18]:
from dimod import BinaryQuadraticModel
from neal import SimulatedAnnealingSampler

# Dados do portfólio
retorno = [0.12, 0.06, 0.08]  # Retorno anual dos ativos
volatilidade = [0.20, 0.08, 0.15]  # Volatilidade anual dos ativos

# Restrições
limite_volatilidade = 0.18  # Limite de volatilidade desejado

# Função objetivo a ser maximizada
def funcao_objetivo(alocacoes):
    return -sum(retorno * alocacoes)

# Restrições
restricoes = (
    {"type": "eq", "fun": lambda alocacoes: sum(alocacoes) - 1},  # Soma das alocações igual a 1
    {"type": "ineq", "fun": lambda alocacoes: limite_volatilidade - sum(volatilidade * alocacoes)}  # Restrição de volatilidade
)

# Limites das alocações (entre 0% e 100% para cada ativo)
limites_alocacao = [(0, 1) for _ in retorno]

# Chute inicial
chute_inicial = [1 / len(retorno) for _ in retorno]

# Otimização
bqm = BinaryQuadraticModel('BINARY')
for i in range(len(retorno)):
    bqm.add_variable(i, -retorno[i])  # Maximizar retorno
    for j in range(i+1, len(retorno)):
        bqm.add_interaction(i, j, 2*limite_volatilidade*volatilidade[i]*volatilidade[j])  # Restrição de volatilidade

# Transformação de QUBO para Ising
ising = bqm.to_ising()

print("Modelo Ising:", ising)


Modelo Ising: ({0: -0.05586, 1: -0.027479999999999997, 2: -0.03622}, {(1, 0): 0.0014399999999999999, (2, 0): 0.0026999999999999997, (2, 1): 0.00108}, -0.12478)


In [19]:
from dimod import BinaryQuadraticModel
from neal import SimulatedAnnealingSampler

# Dados do portfólio
retorno = [0.12, 0.06, 0.08]  # Retorno anual dos ativos
volatilidade = [0.20, 0.08, 0.15]  # Volatilidade anual dos ativos

# Restrições
limite_volatilidade = 0.18  # Limite de volatilidade desejado

# Função objetivo a ser maximizada
def funcao_objetivo(alocacoes):
    return -sum(retorno * alocacoes)

# Restrições
restricoes = (
    {"type": "eq", "fun": lambda alocacoes: sum(alocacoes) - 1},  # Soma das alocações igual a 1
    {"type": "ineq", "fun": lambda alocacoes: limite_volatilidade - sum(volatilidade * alocacoes)}  # Restrição de volatilidade
)

# Limites das alocações (entre 0% e 100% para cada ativo)
limites_alocacao = [(0, 1) for _ in retorno]

# Chute inicial
chute_inicial = [1 / len(retorno) for _ in retorno]

# Otimização
bqm = BinaryQuadraticModel('BINARY')
for i in range(len(retorno)):
    bqm.add_variable(i, -retorno[i])  # Maximizar retorno
    for j in range(i+1, len(retorno)):
        bqm.add_interaction(i, j, 2*limite_volatilidade*volatilidade[i]*volatilidade[j])  # Restrição de volatilidade

# Transformação de QUBO para Ising
ising = bqm.to_ising()

# Transformação de Ising para Hamiltoniano
h = ising[0]
J = ising[1]
offset = ising[2]

# O Hamiltoniano é dado por H = sum(h[i]*s_i) + sum(J[i,j]*s_i*s_j) + offset
# onde s_i são os spins das variáveis.

print("Hamiltoniano:", h, J, offset)


Hamiltoniano: {0: -0.05586, 1: -0.027479999999999997, 2: -0.03622} {(1, 0): 0.0014399999999999999, (2, 0): 0.0026999999999999997, (2, 1): 0.00108} -0.12478


In [22]:
import pennylane as qml
from pennylane import numpy as np

# Dados do portfólio
retorno = np.array([0.12, 0.06, 0.08])  # Retorno anual dos ativos
volatilidade = np.array([0.20, 0.08, 0.15])  # Volatilidade anual dos ativos

# Restrições
limite_volatilidade = 0.18  # Limite de volatilidade desejado

# Função objetivo a ser maximizada
def funcao_objetivo(alocacoes):
    return -np.sum(retorno * alocacoes)

# Restrições
restricoes = (
    {"type": "eq", "fun": lambda alocacoes: np.sum(alocacoes) - 1},  # Soma das alocações igual a 1
    {"type": "ineq", "fun": lambda alocacoes: limite_volatilidade - np.sum(volatilidade * alocacoes)}  # Restrição de volatilidade
)

# Limites das alocações (entre 0% e 100% para cada ativo)
limites_alocacao = [(0, 1) for _ in retorno]

# Chute inicial
chute_inicial = [1 / len(retorno) for _ in retorno]

# Otimização
bqm = BinaryQuadraticModel('BINARY')
for i in range(len(retorno)):
    bqm.add_variable(i, -retorno[i])  # Maximizar retorno
    for j in range(i+1, len(retorno)):
        bqm.add_interaction(i, j, 2*limite_volatilidade*volatilidade[i]*volatilidade[j])  # Restrição de volatilidade

# Transformação de QUBO para Ising
ising = bqm.to_ising()

# Transformação de Ising para Hamiltoniano
h = ising[0]
J = ising[1]
offset = ising[2]

# O Hamiltoniano é dado por H = sum(h[i]*s_i) + sum(J[i,j]*s_i*s_j) + offset
# onde s_i são os spins das variáveis.

# Converter para operador do Pennylane
num_qubits = len(h)
coeffs = []
observables = []
for i in h:
    coeffs.append(h[i])
    observables.append(qml.PauliZ(i))
for i, j in J:
    coeffs.append(J[i, j])
    observables.append(qml.PauliZ(i) @ qml.PauliZ(j))

H = qml.Hamiltonian(coeffs, observables)

# Definir o circuito quântico
dev = qml.device("default.qubit", wires=num_qubits)

@qml.qnode(dev)
def circuit(params):
    for i in range(num_qubits):
        qml.RY(params[i], wires=i)
    return qml.expval(H)

# Definir a função de custo
def cost(params):
    return circuit(params)

# Otimização
opt = qml.GradientDescentOptimizer(stepsize=0.4)
params = np.random.random(num_qubits)  # Inicialização aleatória
for _ in range(100):
    params = opt.step(cost, params)

print("Parâmetros otimizados:", params)
print("Valor mínimo da função de custo:", cost(params))


Parâmetros otimizados: [0.09775426 0.14788122 0.12957608]
Valor mínimo da função de custo: -0.11354842071433371


In [26]:
import pennylane as qml
from pennylane import numpy as np

# Dados do portfólio
retorno = np.array([0.12, 0.06, 0.08])  # Retorno anual dos ativos
volatilidade = np.array([0.20, 0.08, 0.15])  # Volatilidade anual dos ativos

# Restrições
limite_volatilidade = 0.18  # Limite de volatilidade desejado

# Função objetivo a ser maximizada
def funcao_objetivo(alocacoes):
    return -np.sum(retorno * alocacoes)

# Restrições
restricoes = (
    {"type": "eq", "fun": lambda alocacoes: np.sum(alocacoes) - 1},  # Soma das alocações igual a 1
    {"type": "ineq", "fun": lambda alocacoes: limite_volatilidade - np.sum(volatilidade * alocacoes)}  # Restrição de volatilidade
)

# Limites das alocações (entre 0% e 100% para cada ativo)
limites_alocacao = [(0, 1) for _ in retorno]

# Chute inicial
chute_inicial = [1 / len(retorno) for _ in retorno]

# Otimização
bqm = BinaryQuadraticModel('BINARY')
for i in range(len(retorno)):
    bqm.add_variable(i, -retorno[i])  # Maximizar retorno
    for j in range(i+1, len(retorno)):
        bqm.add_interaction(i, j, 2*limite_volatilidade*volatilidade[i]*volatilidade[j])  # Restrição de volatilidade

# Transformação de QUBO para Ising
ising = bqm.to_ising()

# Transformação de Ising para Hamiltoniano
h = ising[0]
J = ising[1]
offset = ising[2]

# O Hamiltoniano é dado por H = sum(h[i]*s_i) + sum(J[i,j]*s_i*s_j) + offset
# onde s_i são os spins das variáveis.

# Converter para operador do Pennylane
num_qubits = len(h)
coeffs = []
observables = []
for i in h:
    coeffs.append(h[i])
    observables.append(qml.PauliZ(i))
for i, j in J:
    coeffs.append(J[i, j])
    observables.append(qml.PauliZ(i) @ qml.PauliZ(j))

H = qml.Hamiltonian(coeffs, observables)

# Definir o circuito quântico
dev = qml.device("default.qubit", wires=num_qubits)

@qml.qnode(dev)
def circuit(params):
    for i in range(num_qubits):
        qml.RX(params[0][i], wires=i)
        qml.RZ(params[1][i], wires=i)
    return qml.expval(H)

# Definir a função de custo
def cost(params):
    return circuit(params)

# Otimização
opt = qml.GradientDescentOptimizer(stepsize=0.4)
params = np.random.random((2, num_qubits))  # Inicialização aleatória
for _ in range(100):
    params = opt.step(cost, params)

print("Parâmetros otimizados:", params)
print("Valor mínimo da função de custo:", cost(params))


Parâmetros otimizados: [[0.11676376 0.27752296 0.1051578 ]
 [0.86884371 0.1140938  0.20064133]]
Valor mínimo da função de custo: -0.11285288411691638
